In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re


def process_text_file(file_path):

  with open(file_path, 'r') as f:
    lines = f.readlines()

  data = [] # create an empty list to store the extracted data
  for line in lines:
    # Usa expressão regular para extrair data, hora, autor e mensagem
    pattern = r"\[(.*?)\] (.*?): (.*)"
    match = re.match(pattern, line)
    if match:
      datetime_str = match.group(1)
      autor = match.group(2)
      mensagem = match.group(3)
      # Separa data e hora
      data_hora = datetime_str.split()
      data_str = data_hora[0] # assign the date string to a new variable data_str
      hora = data_hora[1]
      data.append([data_str, hora, autor, mensagem]) # append the data to the list

  # Cria o DataFrame pandas
  df = pd.DataFrame(data, columns=['data', 'hora', 'autor', 'mensagem'])
  return df

# Caminho do arquivo
file_path = '/content/drive/MyDrive/vite.txt'

# Processa o arquivo e cria o DataFrame
df = process_text_file(file_path)



In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
#transformar o campo data em datetime

df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y', errors='coerce') # specify the format of the datestring and handle errors
#df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y') # specify the format of the datestring and handle errors


In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
# Exibe o DataFrame
#print(df)
#df.head()
data_inicial = input('Inserir data no formato dd/mm/YYYY:  ')
#data_inicial = '01/08/2024'  # Define a data inicial desejada

#inserir no formato americano

df_filtered = df[df['data'] >= data_inicial]
#df_filtered = df[df['data'] >= '01/07/2024']




In [ ]:
df_filtered.info()

In [ ]:
df_filtered.head(30)

In [ ]:
df_filtered.info()

In [ ]:
#mensagens_por_autor = df['autor'].value_counts()
mensagens_por_autor = df_filtered['autor'].value_counts()


#imprimir os 20 maiores do dia 01/08/2024 até 13/09/202413:55
hanking = input('Insira a quantidade de posições:  ')
print('==================================')
print(' ')
print('Serão mostrados os '+ hanking+ ' primeiros a partir de '+ data_inicial)
print(' ')
print('==================================')

# Criar um DataFrame com a contagem de mensagens por autor e a posição
ranking_df = pd.DataFrame({'Autor': mensagens_por_autor.index, 'Mensagens': mensagens_por_autor.values}, index=range(1, len(mensagens_por_autor) + 1))

# Mostrar o ranking com a posição
print(ranking_df.head(int(hanking)))

In [ ]:
 # 1. Número de mensagens por autor
#mensagens_por_autor = df_filtered['autor'].value_counts()
print("Número de mensagens por autor:\n", mensagens_por_autor)
plt.figure(figsize=(100, 50))
#sns.countplot(x='autor', data=df_filtered)
#sns.barplot(x='autor', y=mensagens_por_autor, data=df_filtered, palette='pastel')
#plt.xticks(rotation=45)
#plt.show()
plt.figure(figsize=(100, 50))
plt.bar(mensagens_por_autor.index, mensagens_por_autor.values)
plt.title('Número de mensagens por autor')
plt.xlabel('Autor')
plt.ylabel('Número de mensagens')
plt.tight_layout()
#plt.xticks(rotation=45)
plt.show()

In [ ]:
# 2. Distribuição de mensagens ao longo do tempo
# Remove brackets and any leading/trailing spaces
#df['data'] = df['data'].str.strip('[] ').replace('', pd.NA)
# Convert to datetime, handling errors by setting invalid dates to NaT
df_filtered['data'] = pd.to_datetime(df_filtered['data'], format='%d/%m/%Y %H:%M:%S', errors='coerce')
# Remove rows with NaT in 'data' column
df = df_filtered.dropna(subset=['data'])
mensagens_por_data = df_filtered.groupby(df['data'].dt.date)['mensagem'].count()
plt.figure(figsize=(20, 10))
plt.plot(mensagens_por_data.index, mensagens_por_data.values)
plt.title('Distribuição de mensagens ao longo do tempo')
plt.xlabel('Data')
plt.ylabel('Número de mensagens')
plt.xticks(rotation=45)
plt.show()



In [ ]:
# 3. Palavras mais frequentes
from collections import Counter
todas_as_palavras = []
for mensagem in df_filtered['mensagem']:
  palavras = mensagem.split()
  todas_as_palavras.extend(palavras)
contagem_palavras = Counter(todas_as_palavras)
palavras_mais_frequentes = contagem_palavras.most_common(200)
print("Palavras mais frequentes:\n", palavras_mais_frequentes)

In [ ]:
# prompt: desejo realizar análise de sentimentos

!pip install textblob

from textblob import TextBlob

# Função para analisar o sentimento da mensagem
def analisar_sentimento(texto):
  analysis = TextBlob(texto)
  if analysis.sentiment.polarity > 0:
    return 'positivo'
  elif analysis.sentiment.polarity < 0:
    return 'negativo'
  else:
    return 'neutro'

#prompt: inserir data no formato dd/mm/YYYY:
#data_inicial = input('Inserir data no formato dd/mm/YYYY:  ')
#df_filtered = df[df['data'] >= str(data_inicial)]

#mensagens_por_autor = df['autor'].value_counts()
#mensagens_por_autor = df_filtered['autor'].value_counts()


# Aplicar a função de análise de sentimento às mensagens
df_filtered['sentimento'] = df_filtered['mensagem'].apply(analisar_sentimento)

# Analisar a distribuição de sentimentos
sentimentos_por_autor = df_filtered.groupby('autor')['sentimento'].value_counts()
#sentimentos_por_autor = df_filtered.groupby('autor')[df_filtered['sentimento']=='negativo'].value_counts()
print("Sentimentos por autor:\n", sentimentos_por_autor)

# Visualizar a distribuição de sentimentos
plt.figure(figsize=(10, 5))
sns.countplot(x='sentimento', data=df_filtered)
plt.title('Distribuição de sentimentos')
plt.xlabel('Sentimento')
plt.ylabel('Número de mensagens')
plt.show()

In [ ]:
#df_filtered[df_filtered['sentimento']=='positivo'].head(50)
df_filtered[df_filtered['sentimento']=='negativo'].head(50)

In [ ]:
print(df_filtered[df_filtered['autor']=='Jonh Selmo'].groupby('autor')['sentimento'].value_counts())